In [1]:
%pip install --upgrade pip
%pip install cmake

     ---------------------------------------- 1.8/1.8 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/37.6 MB ? eta -:--:--
   -- ------------------------------------- 2.4/37.6 MB 12.2 MB/s eta 0:00:03
   ----- ---------------------------------- 4.7/37.6 MB 11.9 MB/s eta 0:00:03
   ------- -------------------------------- 7.3/37.6 MB 11.9 MB/s eta 0:00:03
   ---------- ----------------------------- 9.7/37.6 MB 11.8 MB/s eta 0:00:03
   ------------- -------------------------- 12.3/37.6 MB 11.9 MB/s eta 0:00:03
   --------------- ------------------------ 14.7/37.6 MB 11.8 MB/s eta 0:00:02
   ------------------ --------------------- 17.3/37.6 MB 11.8 MB/s eta 0:00:02
   --------------------- ------------------ 19.9/37.6 MB 11.9 MB/s eta 0:00:02
   -----------

In [2]:
%pip install opencv-python opencv_contrib-python
%pip install face_recognition
%pip install numpy 

Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
      -------------------------------------- 2.4/100.1 MB 12.2 MB/s eta 0:00:09
     - ------------------------------------- 4.7/100.1 MB 11.9 MB/s eta 0:00:09
     -- ------------------------------------ 7.3/100.1 MB 11.9 MB/s eta 0:00:08
     --- ---------------------------------- 10.0/100.1 MB 11.9 MB/s eta 0:00:08
     ---- --------------------------------- 12.3/100.1 MB 11.9 MB/s eta 0:00:08
     ----- -------------------------------- 14.9/100.1 MB 11.9 MB/s eta 0:00:08
     ------ ------------------------------- 17.3/100.1 MB 11.8 MB/s eta 0:00:07
     ------- ------------------------------ 19.9/100.1 MB 11.9 MB/s eta 0:00:07
     -------- ----------------------------- 22.3/100.1 MB 11.8 MB/s eta 0:00:07
     --------- ---------------------------- 24.9/100.1 MB 11.9 MB/s eta 0:00:07
     ---------- --------------------------- 27.5/100.1 MB 11.

In [2]:
import cv2
import face_recognition
import os, numpy as np

In [6]:
#Directory to store training images
os.makedirs('training_faces', exist_ok=True)
os.makedirs('encodings', exist_ok=True)

Save faces

In [10]:

def capture_images():
    #Name for the captured images
    name = input("Enter name: ")

    #Create a folder for person if it doesnt exist yet
    person_folder = f"training_faces/{name}"
    os.makedirs(person_folder, exist_ok= True)


    #Initialize video capture
    cap = cv2.VideoCapture(0)

    #Frames Loop
    while True:
        success, frame = cap.read()
        if not success:
            print("Failed to capture frame")
            break

        cv2.imshow("Training - Press 'c' to capture, 'r' to restart or 'q' to quit", frame)

        #Capture the frame when 'c' is pressed
        if cv2.waitKey(1) & 0xFF == ord('c'):
            img_path = f'training_faces/{name}/{name}_{len(os.listdir(person_folder))+1}.jpg'
            cv2.imwrite(img_path, frame) #Save the original frame
            print(f"Image saved at: {img_path}")

        #Exit loop with when pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if cv2.waitKey(1) & 0xFF ==ord('r'):
            cap.release()
            cv2.destroyAllWindows()
            return capture_images()

    cap.release()
    cv2.destroyAllWindows()


capture_images()


Image saved at: training_faces/JoseLuis/JoseLuis_1.jpg
Image saved at: training_faces/JoseLuis/JoseLuis_2.jpg
Image saved at: training_faces/JoseLuis/JoseLuis_3.jpg
Image saved at: training_faces/Mafer/Mafer_1.jpg


Encode and train based on faces

In [11]:
def encode():
    training_dir = "training_faces"

    output_dir = "encodings"
    #Just in case folder doesnt exist
    os.makedirs(output_dir, exist_ok=True)

    for person_name in os.listdir(training_dir):
        person_folder = os.path.join(training_dir, person_name)


        #If folder doesnt exist
        if not os.path.isdir(person_folder):
            continue

        print(f"Processing images for {person_name}")


        all_encodings = []

          
        #Go trough each image in a folder
        for filename in os.listdir(person_folder):
            if filename.lower().endswith((".jpg",".jpeg", ".png")):
                img_path = os.path.join(person_folder, filename)

                #Load image
                image = cv2.imread(img_path)
                if image is None:
                    print(f" Cannot read {img_path}")
                    continue

                img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                #Get face encodings
                encodings = face_recognition.face_encodings(img_rgb)
                if encodings:
                    all_encodings.append(encodings[0])
                else:
                    print(f"No face found in file {filename}")

        #Save encodings for that one person
        if all_encodings:
            np.save(os.path.join(output_dir, f"{person_name}_encodings.npy"), np.array(all_encodings))
            print(f"Saved {len(all_encodings)} encodings for {person_name}")
        else:
            print(f"No encodings fouind for {person_name}")

    print("Training has been completed.")


encode()

Processing images for JoseLuis
Saved 3 encodings for JoseLuis
Training has been completed.


Load Encodings

In [13]:
def load_encodings(encodings_path):
    encodings = []
    class_names = []
    for file in os.listdir(encodings_path):
        if file.endswith("_encodings.npy"):
            class_name = file.split('_')[0]
            encoding = np.load(os.path.join(encodings_path, file))
            encodings.append(encoding)
            class_names.append(class_name)
    return encodings, class_names

In [ ]:
#Path for saved encodings
encodings_path = 'encodings'
known_encodings, class_names = load_encodings(encodings_path)
print(f"Loaded classes:{class_names}")

Loaded classes:['JoseLuis']


In [ ]:
cap = cv2.VideoCapture(0)
scale = 0.25
cap = cv2.VideoCapture(0)


In [ ]:
def start_program(cap, known_encodings):
    while True:
        success, frame = cap.read()
        if not success:
            print("Failed to capture frame")
            break
        rgb_frame =cv2.cvtColor(frame, cv2.COLOR_BAYER_BG2RGB)

        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_encoding, face_loaction in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.6)
            name ="Unkown"

            if True in matches:
                first_match_index = matches.index(True)